In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy.units as u # I use astropy units in everything, very helpful for bookkeeping/debugging

from lim import lim # The base intensity mapping object

matplotlib.rcParams.update({'font.size': 18,'figure.figsize':[8,7]}) # Make figures better
#matplotlib.rcParams,update({}'figure.figsize'] = [8,7]
from scipy.ndimage import gaussian_filter

In [2]:
data = np.load('nate_sims/new_nate_lightcone.npz', allow_pickle = True)

In [3]:
data['peak']

array([[-3.82396355e+01,  2.36126694e+02,  6.73134668e+03, ...,
         2.34500000e+02,  6.72850000e+03,  5.92143583e+00],
       [-4.88217560e+02,  9.98937531e+01,  6.81045850e+03, ...,
         9.75000000e+01,  6.81450000e+03,  6.41378117e+00],
       [-7.57260818e+01,  4.37949249e+02,  6.85425244e+03, ...,
         4.33500061e+02,  6.85350000e+03,  6.40876436e+00],
       ...,
       [ 4.32893646e+02, -1.27911749e+01,  6.49955713e+03, ...,
        -1.25000000e+01,  6.49950000e+03,  6.33086157e+00],
       [ 4.37036163e+02, -1.24819355e+01,  6.49879443e+03, ...,
        -1.25000000e+01,  6.49950000e+03,  6.05224180e+00],
       [ 3.24653351e+02, -7.62369251e+00,  6.49859229e+03, ...,
        -6.50000000e+00,  6.49950000e+03,  6.49030733e+00]], dtype=float32)